# Problem set 6: Solving the Solow model

In [1]:
import numpy as np
from scipy import linalg
from scipy import optimize
import sympy as sm

# Tasks

## Solving matrix equations I

In [2]:
np.random.seed(1900)
n = 5
A = np.random.uniform(size=(n,n))
b = np.random.uniform(size=n)
c = np.random.uniform(size=n)
d = np.random.uniform(size=n)

**Question A:** Find the determinant of $[A \cdot A]^{-1}$

In [15]:
# write your code here

X = linalg.det(linalg.inv(A@A))
print(X)


13132.548734458798


**Answer:** see A1.py

**Question B:** Solve the following equation systems directly using **scipy**.

$$
\begin{aligned}
Ax &= b \\
Ax &= c \\
Ax &= d 
\end{aligned}
$$

**Answer:** A2.py

In [17]:
xb = linalg.solve(A,b)
xc = linalg.solve(A,c)
xd = linalg.solve(A,d)

for i in [xb, xc, xd]:
    print(i)

[-15.33189031 -24.00998148  40.02675108  15.24193293   4.89008792]
[ -7.72469812 -11.6354722   20.86796131   7.93343031   1.55909328]
[-2.57374569 -4.00256301  7.93409587  2.56007481  1.25709881]


**Question C:** Solve the same equation systems as above using `linalg.lu_factor()` and `linalg.lu_solve()`. What is the benefit of this approach?

In [20]:
LU,piv = linalg.lu_factor(A) 

xb2 = linalg.lu_solve((LU,piv),b)
xc2 = linalg.lu_solve((LU,piv),c)
xd2 = linalg.lu_solve((LU,piv),d)

for i in [xb2, xc2, xd2]:
    print(i)


[-15.33189031 -24.00998148  40.02675108  15.24193293   4.89008792]
[ -7.72469812 -11.6354722   20.86796131   7.93343031   1.55909328]
[-2.57374569 -4.00256301  7.93409587  2.56007481  1.25709881]


**Answer:** A3.py

## Solving matrix equations II

In [21]:
F = np.array([[2.0, 1.0, -1.0], [-3.0, -1.0, 2], [-2.0, 1.0, 2.0]])
e = np.array([8.0, -11.0, -3.0])

**Question:** Use the function `gauss_jordan()` in the `numecon_linalg` module located in this folder to solve

$$
Fx = e
$$

In [47]:
# write your code here
import numecon_linalg as econ

Y = np.column_stack((F,e))

econ.gauss_jordan(Y)


assert np.allclose(F@Y[:,-1],e)


**Answer:** see A4.py

## Symbolic

**Question A:** Find

$$
\lim_{x \rightarrow 0} \frac{\sin(x)}{x}
$$

and

$$
\frac{\partial\sin(2x)}{\partial x} 
$$

In [60]:
# write your code here

x = sm.symbols('x')
sm.limit(sm.sin(x)/x,x,0)

sm.diff(sm.sin(2*x),x)

2*cos(2*x)

**Answer:** A5.py

**Question B:** Solve the equation

$$ 
\frac{\sin(x)}{x} = 0
$$

In [67]:
# write your code here

sm.solve(sm.sin(x)/x)

[pi]

**Answer:** A6.py

# Problem: Solve the Solow model

## Introduction

Consider the **standard Solow-model** where:

1. $K_t$ is capital2
2. $L_t$ is labor (growing with a constant rate of $n$)
3. $A_t$ is technology (growing with a constant rate of $g$)
4. $Y_t = F(K_t,A_tL_t)$ is GDP

**Saving** is a constant fraction of GDP

$$ 
S_t = sY_t,\,s\in(0,1)
$$

such that **capital accumulates** according to

$$
K_{t+1}=S_{t}+(1-\delta)K_{t}=sF(K_{t},A_{t}L_{t})+(1-\delta)K_{t}, \delta \in (0,1)
$$

The **production function** has **constant-return to scale** such that

$$
\frac{Y_{t}}{A_{t}L_{t}}=\frac{F(K_{t},A_{t}L_{t})}{A_{t}L_{t}}=F(\tilde{k}_{t},1)\equiv f(\tilde{k}_{t})
$$

where $\tilde{k}_t = \frac{K_t}{A_{t}L_{t}}$ is the technology adjusted capital-labor ratio.

The **transition equation** then becomes

$$
\tilde{k}_{t+1}= \frac{1}{(1+n)(1+g)}[sf(\tilde{k}_{t})+(1-\delta)\tilde{k}_{t}]
$$

If the **production function** is **Cobb-Douglas** then

$$
F(K_{t},A_{t}L_{t})=K_{t}^{\alpha}(A_{t}L_{t})^{1-\alpha}\Rightarrow f(\tilde{k}_{t})=\tilde{k}_{t}^{\alpha}
$$

If it is **CES** (with $\beta < 1, \beta \neq 0$) then

$$
F(K_{t},A_{t}L_{t})=(\alpha K_{t}^{\beta}+(1-\alpha)(A_{t}L_{t})^{\beta})^{\frac{1}{\beta}}\Rightarrow f(\tilde{k}_{t})=(\alpha\tilde{k}_{t}^{\beta}+(1-\alpha))^{\frac{1}{\beta}}
$$

## Steady state

Assume the production function is **Cobb-Douglas**.

**Question A:** Use **sympy** to find an analytical expression for the steady state, i.e. solve

$$
\tilde{k}^{\ast}= \frac{1}{(1+n)(1+g)}[sf(\tilde{k}^{\ast})+(1-\delta)\tilde{k}^{\ast}]
$$

In [68]:
k = sm.symbols('k')
alpha = sm.symbols('alpha')
delta = sm.symbols('delta')
s = sm.symbols('s')
g = sm.symbols('g')
n = sm.symbols('n')

In [85]:
# write your code here

f = k**alpha
ss = sm.Eq(k, (s*f+(1-delta)*k)/((1+n)*(1+g)))

kss = sm.solve(ss,k)[0]

kss

((delta + g*n + g + n)/s)**(1/(alpha - 1))

**Answer:** see A7.py

**Question B:** Turn you solution into a Python function called as `ss_func(s,g,n,delta,alpha)`. 

In [92]:
# write your code here

ss_func = sm.lambdify((s,g,n,delta,alpha),kss)

ss_func(0.2,0.02,0.01,0.1,1/3)

1.903831539231319

**Answer:** A8.py

**Question C**: Find the steady state numerically using root-finding with `optimize.root_scalar`.

In [100]:
s = 0.2
g = 0.02
n = 0.01
alpha = 1/3
delta = 0.1

# write your code here

f = lambda k: k**alpha

lambda_kss = lambda kss: kss - (s*f(kss)+(1-delta)*kss)/((1+n)*(1+g))
result = optimize.root_scalar(lambda_kss,bracket=[0.1,100],method='brentq')

result

      converged: True
           flag: 'converged'
 function_calls: 10
     iterations: 9
           root: 1.9038315392313185

**Answer:** A9.py

**Question D:** Now assume the production function is CES. Find the steady state for $k$ for the various values of $\beta$ shown below.

In [23]:
betas = [-0.5,-0.25,-0.1,-0.05,0.05,0.1,0.25,0.5]

# write your code here

**Answer:** A10.py   